<a href="https://colab.research.google.com/github/Clebiotalves/pyton/blob/master/C%C3%B3pia_de_SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [23]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[4, 5, 6]


In [26]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[]


## Mudanças feitas
Como o professor liberou, decidi criar uma função que, automaticamente, com base na descrição, envie uma mensagem correspondente.

## Criando uma função mensagem

In [27]:
def message_user(user):
  if user['description'] == "PIX":
    print("Com PIX Santander, suas transações são instantâneas, assim como a praticidade e segurança que só o Santander oferece. Simplifique sua vida financeira conosco!")

  elif user['description'] == "Pagar":
    print("O Santander tem soluções de crédito sob medida pra você. Confira!")

  elif user['description'] == "Transferir":
    print("Transfira com confiança, transfira com Santander. Sua segurança financeira em cada movimento, porque no Santander, cuidamos do seu dinheiro como se fosse nosso.")

  elif user['description'] == "Conta Corrente":
    print(
"Na conta corrente Santander, a praticidade e a segurança andam lado a lado. Gerencie seu dinheiro com facilidade e conte com a solidez de uma parceria que valoriza cada centavo. Faça da sua conta corrente, a conta que faz a diferença, faça Santander.")

  elif user['description'] == "Cartões":
    print("Santander Seguro Casa, seu faz-tudo. Mais de 50 serviços pra você. Confira!")

  else:
    print("Sejá Benvindo")



In [28]:
for user in users:
  news = message_user(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": user['description']
      })


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [29]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")